In [216]:
#%matplotlib inline
import os
import numpy as np
from PIL import Image
import matplotlib as mp
from matplotlib import pyplot,pylab
plt = pyplot
import scipy
from __future__ import division

import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
import string
import pandas as pd
from scipy import stats

%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs

import json
import pymongo as pm

from svgpathtools import parse_path

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

#### set up access to db 

In [217]:
#### set vars 
import pandas as pd
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]

user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
category_flag = 'chairs4'
dbname = 'stimuli'
colname = 'svg_annotation_sketchpad_basic_{}_dev'.format(category_flag)
db = conn[dbname]
coll = db[colname]

#### how to filter for a particular thing using pymongo

In [218]:
## get me a record (usually it will be deterministic, and get the "first one")
coll.find_one()

{u'Distractor1': u'leather',
 u'Distractor2': u'inlay',
 u'Distractor3': u'squat',
 u'_id': ObjectId('5b64eea2938d5c3f0c54461b'),
 u'category': u'chair',
 u'condition': u'closer',
 u'drawDuration': 21.246,
 u'gameID': u'7004-6ff0964c-ff95-40f9-8696-d6a8931c94d5',
 u'games': [],
 u'index': 8,
 u'iteration': u'pilot2',
 u'iterationName': u'sketchpad_basic_chairs4',
 u'level_0': 1,
 u'mean_intensity': 0.0722,
 u'numSplines': 52,
 u'numStrokes': 11,
 u'outcome': True,
 u'parts': [u'backrest,armrest,seat,leg'],
 u'pose': 35,
 u'response': u'waiting',
 u'svg': [u'M65,288c0,-47.26597 -10.61179,-88.33968 29,-123c7.48469,-6.5491 16.56343,-16 27,-16',
  u'M70,235c29.90144,0 59.48333,3 89,3c1.59397,0 8.92315,1.07685 10,0c3.89674,-3.89674 42.96756,-41 37,-41',
  u'M68,231c11.52976,0 51.77736,-44.77736 59,-52c3.47514,-3.47514 59.1498,22 76,22',
  u'M126,148c0,9.3314 -1.24095,22.5181 3,31',
  u'M152,180c0,-5.00908 4.70884,-10.87349 3,-16c-3.19002,-9.57006 -11.78009,-14.56017 -16,-23c-1.5706,-3.14121

In [105]:
## filter based on a key
## so in this example, we'll just get the "waiting" chair sketches
## when it's a string you can just put it on the other side of the colon
##stims = coll.find({'target':'waiting'}).sort('gameID')
stims = coll.find().sort('gameID')

In [106]:
## how many of these stims are there in the db?
stims.count()

371

In [107]:
## build up list of svg strings for each waiting chair sketch
svgs = []
for rec in stims:
    svgs.append(rec['svg'])

    

In [108]:
len(svgs)

371

In [109]:
splines=[]
for i in range(len(svgs)):
    for j in range(len(svgs[i])):
        splines.append(parse_path(svgs[i][j]))

In [151]:
splines

[Path(CubicBezier(start=(152+69j), control1=(152+98.15975j), control2=(151+127.62575j), end=(151+157j))),
 Path(CubicBezier(start=(211+68j), control1=(211+91.38762j), control2=(212.11733+115.70932j), end=(210+139j)),
      CubicBezier(start=(210+139j), control1=(208.931+150.75899j), control2=(206+162.8136j), end=(206+174j)),
      CubicBezier(start=(206+174j), control1=(206+175.33333j), control2=(206+175.33333j), end=(206+174j))),
 Path(CubicBezier(start=(151+153j), control1=(151+164.35574j), control2=(73+183.79654j), end=(73+196j)),
      CubicBezier(start=(73+196j), control1=(73+197.79505j), control2=(76.30728+197.40257j), end=(78+198j)),
      CubicBezier(start=(78+198j), control1=(90.71151+202.48641j), control2=(103.93761+205.7344j), end=(117+209j)),
      CubicBezier(start=(117+209j), control1=(159.52471+219.63118j), control2=(173.60025+241.86633j), end=(208+196j)),
      CubicBezier(start=(208+196j), control1=(209.94541+193.40612j), control2=(225+184.34021j), end=(225+184j)),
   

In [219]:
import svgpathtools
numLines=0
for i in range(len(splines)):
    if type(splines[i][0])!= svgpathtools.path.CubicBezier:
        numLines+=1
        
  

In [201]:
numLines

10812

In [67]:
this_sketch

[u'M152,69c0,29.15975 -1,58.62575 -1,88',
 u'M211,68c0,23.38762 1.11733,47.70932 -1,71c-1.069,11.75899 -4,23.8136 -4,35c0,1.33333 0,1.33333 0,0',
 u'M151,153c0,11.35574 -78,30.79654 -78,43c0,1.79505 3.30728,1.40257 5,2c12.71151,4.48641 25.93761,7.7344 39,11c42.52471,10.63118 56.60025,32.86633 91,-13c1.94541,-2.59388 17,-11.65979 17,-12c0,-15.482 -67.1772,-21 -79,-21c-2.09271,0 -8,0.93187 -8,-2']

In [94]:
this_stroke = this_sketch[0]

In [95]:
this_stroke

u'M152,69c0,29.15975 -1,58.62575 -1,88'

In [96]:
## it parses the stroke into constituent splines AND converts them to absolute coordinates
parsed_stroke = parse_path(this_stroke)